In [27]:
import math
from datetime import datetime
from pathlib import Path
from astropy.io import fits

In [36]:
rootdir = Path("/Volumes/LaCie 8TB/wfsdat/20180510")

In [37]:
files = sorted(list(rootdir.glob("mmirs*.fits")))

In [38]:
timedict = {}
for f in files:
    with fits.open(f) as hdulist:
        hdr = hdulist[-1].header
        data = hdulist[-1].data
    timedict[str(f)] = hdr['DATE-OBS']
print(timedict)

In [42]:
sec = 0.
for i in range(0, len(files)):
    if i < len(files)-1:
        t1 = datetime.strptime(timedict[str(files[i])], "%Y-%m-%dT%H:%M:%S")
        t2 = datetime.strptime(timedict[str(files[i+1])], "%Y-%m-%dT%H:%M:%S")
    else:  # handle last file
        t1 = datetime.strptime(timedict[str(files[i-1])], "%Y-%m-%dT%H:%M:%S")
        t2 = datetime.strptime(timedict[str(files[i])], "%Y-%m-%dT%H:%M:%S")
    diff = t2-t1
    
    # exposure times are almost always in multiples of 5 sec unless the exposures are very short
    diff_sec = 5 * math.floor(diff.seconds/5)
    
    # mmirs wfs exposures should almost never be more than 3 min during normal operations.
    # large gaps are assumed to be the end of a track so 200 seems a good cutoff to reject
    # those and use the previous time diff instead.
    if diff_sec < 200:
        sec = diff_sec
    
    f = files[i]
    with fits.open(f) as hdulist:
        changed = False
        for h in hdulist:
            if 'EXPTIME' in h.header:
                if h.header['EXPTIME'] == 0.0:
                    print(f"Setting EXPTIME to {sec} in {str(f)}..")
                    h.header['EXPTIME'] = sec
                    changed = True
        if changed:
            hdulist.writeto(f, overwrite=True)

Setting EXPTIME to 30 in /Volumes/LaCie 8TB/wfsdat/20180510/mmirs_wfs_0553.fits..


In [20]:
diff.second

4

In [28]:
5 * math.floor(diff.seconds/5)

175